**Step 1:** Mount your Google Drive by clicking on "Mount Drive" in the Files section (panel to the left of this text.)

**Step 2:** Go to Runtime -> Change runtime type and select TPU.

**Step 3:** Create a folder in your Google Drive, and rename it to "vMalConv"

**Step 4:** Download the pre-processed training and test datasets.

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
!wget https://sagemaker-malconv.s3.amazonaws.com/sagemaker/X_train.dat
!wget https://sagemaker-malconv.s3.amazonaws.com/sagemaker/X_test.dat
!wget https://sagemaker-malconv.s3.amazonaws.com/sagemaker/y_train.dat
!wget https://sagemaker-malconv.s3.amazonaws.com/sagemaker/y_test.dat
!wget https://sagemaker-malconv.s3.amazonaws.com/sagemaker/metadata.csv

--2020-05-02 01:19:11--  https://sagemaker-malconv.s3.amazonaws.com/sagemaker/X_train.dat
Resolving sagemaker-malconv.s3.amazonaws.com (sagemaker-malconv.s3.amazonaws.com)... 52.217.43.12
Connecting to sagemaker-malconv.s3.amazonaws.com (sagemaker-malconv.s3.amazonaws.com)|52.217.43.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8571600000 (8.0G) [binary/octet-stream]
Saving to: ‘X_train.dat.1’

X_train.dat.1       100%[===================>]   7.98G  58.2MB/s    in 2m 1s   

2020-05-02 01:21:12 (67.5 MB/s) - ‘X_train.dat.1’ saved [8571600000/8571600000]

--2020-05-02 01:21:15--  https://sagemaker-malconv.s3.amazonaws.com/sagemaker/X_test.dat
Resolving sagemaker-malconv.s3.amazonaws.com (sagemaker-malconv.s3.amazonaws.com)... 52.216.224.200
Connecting to sagemaker-malconv.s3.amazonaws.com (sagemaker-malconv.s3.amazonaws.com)|52.216.224.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1904800000 (1.8G) [binary/octet-stream]
Savin

**Step 5:** Copy the downloaded files to vMalConv

In [0]:
!cp /content/X_train.dat /content/drive/My\ Drive/vMalConv/X_train.dat
!cp /content/X_test.dat /content/drive/My\ Drive/vMalConv/X_test.dat
!cp /content/y_train.dat /content/drive/My\ Drive/vMalConv/y_train.dat
!cp /content/y_test.dat /content/drive/My\ Drive/vMalConv/y_test.dat
!cp /content/metadata.csv /content/drive/My\ Drive/vMalConv/metadata.csv

**Step 6:** Download and install Ember:

In [36]:
!wget https://github.com/endgameinc/ember/archive/master.zip
!unzip master.zip
!rm master.zip
!cp -r ember-master/* .
!rm -r ember-master
!pip install -r requirements.txt
!python setup.py install

--2020-05-02 01:25:29--  https://github.com/endgameinc/ember/archive/master.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/endgameinc/ember/zip/master [following]
--2020-05-02 01:25:29--  https://codeload.github.com/endgameinc/ember/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [   <=>              ]  11.22M  13.0MB/s    in 0.9s    

2020-05-02 01:25:30 (13.0 MB/s) - ‘master.zip’ saved [11769324]

Archive:  master.zip
f9a018632ba108b4e25c33d6c3e2e7a7c4487f58
   creating: ember-master/
  inflating: ember-master/LICENSE.txt  
  inflating: ember-master/README.md  
   creating:

**Step 7:** Read vectorized features from the data files.

In [37]:
import ember
X_train, y_train, X_test, y_test = ember.read_vectorized_features("drive/My Drive/vMalConv/")
metadata_dataframe = ember.read_metadata("drive/My Drive/vMalConv/")

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


**Step 7:** Get rid of rows with no labels.

In [0]:
labelrows = (y_train != -1)
X_train = X_train[labelrows]
y_train = y_train[labelrows]

In [0]:
import h5py
h5f = h5py.File('X_train.h5', 'w')
h5f.create_dataset('X_train', data=X_train)
h5f.close()
h5f = h5py.File('y_train.h5', 'w')
h5f.create_dataset('y_train', data=y_train)
h5f.close()

In [0]:
!cp /content/X_train.h5 /content/drive/My\ Drive/vMalConv/X_train.h5
!cp /content/y_train.h5 /content/drive/My\ Drive/vMalConv/y_train.h5

> **Exercise 1:** Complete the following code to create the architecture of MalConv in Keras:

In [0]:
def make_model():
  import tensorflow as tf
  from tensorflow import keras
  from tensorflow.keras import layers
  feature_size=2381
  tf.compat.v1.disable_eager_execution()

  keras.backend.clear_session()
  
  # Model architecture
  from tensorflow.keras import layers
  model = tf.keras.Sequential()
  ### Your code -- Define the layers of MalConv ###
  model.add(layers.InputLayer(input_shape=(1,feature_size)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(1500, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation='sigmoid'))

  model.compile(tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])
  print(model.summary())
  return model
 

  model.compile(tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])
  print(model.summary())
  return model

In [42]:
model = make_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 1, 2381)           0         
_________________________________________________________________
dense (Dense)                (None, 1, 1500)           3573000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1500)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              1501      
Total params: 3,574,501
Trainable params: 3,574,501
Non-trainable params: 0
_________________________________________________________________
None


**Step 8:** Partial fit the standardScaler to avoid overloading the memory:

In [0]:
from sklearn.preprocessing import StandardScaler
mms = StandardScaler()
for x in range(0,600000,100000):
  mms.partial_fit(X_train[x:x+100000])

In [0]:
X_train = mms.transform(X_train)

In [0]:
## Reshape to create 3 channels ##
import numpy as np
X_train = np.reshape(X_train,(-1,1,2381))
y_train = np.reshape(y_train,(-1,1,1))

> **Exercise 2:** Complete the following code to train the model for 30 epochs, with a batch size of 128, and 20% validation split. 

In [46]:
%tensorflow_version 2.x
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

save_dir = "drive/My Drive/vMalConv/"

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

### Your code ###

model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
          loss='binary_crossentropy',
          metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])

history = model.fit(X_train, y_train,
                batch_size=128,
                epochs=30,
                  validation_split=.2,
                  callbacks=[callback]
                  )
# Save the weights #
model.save_weights (save_dir+'weights.h5')

# Save the model architecture #
model_json = model.to_json()
with open(save_dir+"model.json", "w") as json_file:
    json_file.write(model_json)

print("model saved.")

Train on 480000 samples, validate on 120000 samples
Epoch 1/30
480000/480000 [==============================] - 73s 153us/sample - loss: 6.5256 - accuracy: 0.9404 - auc_1: 0.9464 - precision_1: 0.9393 - val_loss: 14.4840 - val_accuracy: 0.9603 - val_auc_1: 0.9640 - val_precision_1: 0.9873
Epoch 2/30
480000/480000 [==============================] - 68s 142us/sample - loss: 9.6596 - accuracy: 0.9589 - auc_1: 0.9622 - precision_1: 0.9580 - val_loss: 2.8900 - val_accuracy: 0.9820 - val_auc_1: 0.9853 - val_precision_1: 0.9797
Epoch 3/30
480000/480000 [==============================] - 63s 132us/sample - loss: 9.8723 - accuracy: 0.9630 - auc_1: 0.9666 - precision_1: 0.9620 - val_loss: 10.9019 - val_accuracy: 0.9760 - val_auc_1: 0.9771 - val_precision_1: 0.9749
Epoch 4/30
480000/480000 [==============================] - 56s 117us/sample - loss: 9.2982 - accuracy: 0.9658 - auc_1: 0.9696 - precision_1: 0.9644 - val_loss: 4.5520 - val_accuracy: 0.9860 - val_auc_1: 0.9893 - val_precision_1: 0.992

**Final Steps:** Download this Jupyter Notebook, and the saved model files.